# Distilling Knowledge in Multiple Students  using Generative models

In [1]:
# !nvidia-smi

In [2]:
import numpy as np
import tensorflow as tf
import tensorflow.keras
import tensorflow.keras.backend as K
from tensorflow.keras.datasets import fashion_mnist,mnist,cifar10
from tensorflow.keras.layers import Conv2D,Activation,BatchNormalization,UpSampling2D,Embedding,ZeroPadding2D, Input, Flatten, Dense, Reshape, LeakyReLU, Dropout,MaxPooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras import regularizers

from tensorflow.keras.utils import Progbar
from keras.initializers import RandomNormal
import random
from sklearn.model_selection import train_test_split
from tensorflow.keras import utils as np_utils

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

### Teacher's Training

In [3]:
#Loading and splitting the dataset into train, validation and test
nb_classes = 10

(X_Train, y_Train), (X_test, y_test) = mnist.load_data()
X_train, X_val, y_train, y_val = train_test_split(X_Train, y_Train, test_size=0.20)
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_val = np_utils.to_categorical(y_val, nb_classes)

X_train = X_train.reshape(48000, 28, 28, 1)
X_val = X_val.reshape(12000, 28, 28, 1)

X_train = X_train.astype('float32')
X_val = X_val.astype('float32')

# Normalize the values
X_train /= 255
X_val /= 255


#Creating a teacher network
input_shape = (28, 28, 1) # Input shape of each image

teacher = Sequential()
teacher.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
teacher.add(Conv2D(64, (3, 3), activation='relu'))
teacher.add(MaxPooling2D(pool_size=(2, 2)))

teacher.add(Dropout(0.25)) # For reguralization

teacher.add(Flatten())
teacher.add(Dense(256, activation='relu'))
teacher.add(Dense(256, activation='relu', name="dense_1"))

teacher.add(Dropout(0.5)) # For reguralization

teacher.add(Dense(nb_classes, name = 'dense_2'))
teacher.add(Activation('softmax')) # Note that we add a normal softmax layer to begin with

teacher.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(teacher.summary())


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               2359552   
_________________________________________________________________
dense_1 (Dense)              (None, 256)               6

In [4]:
#  Train the teacher model as usual
epochs = 0
batch_size = 256
teacher.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_val, Y_val))


### Evaluation 

In [5]:
Y_test = np_utils.to_categorical(y_test, nb_classes)

In [6]:
X_test = X_test.reshape(10000, 28, 28, 1)
X_test = X_test.astype('float32')
# Normalize the values
X_test /= 255

In [7]:
teacher.load_weights("Teacher_MNIST_98.h5")

In [8]:
teacher.evaluate(X_test,Y_test)

313/313 [==============================] - 2s 3ms/step - loss: 0.0484 - accuracy: 0.9874


[0.03833923488855362, 0.9894999861717224]

### Dense Vector split

In [9]:
teacher_WO_Softmax = Model(teacher.input, teacher.get_layer('dense_1').output)
train_dense = teacher_WO_Softmax.predict(X_train)
val_dense = teacher_WO_Softmax.predict(X_val)


# 1 Student case
# ---------------------------------------------
s1Train=train_dense[:,:256]

## GANs' Training

In [10]:
# import np.random import random
BATCH_SIZE=32
def smooth_real_labels(y):
    return y - 0.3+(np.random.random(y.shape)*0.5)
def smooth_fake_labels(y):
    return y + (0.3 * np.random.random(y.shape))
def build_gan(gen,disc): 
    disc.trainable = False
    input= Input(shape=input_shape)
    output = gen(input)
    output2= disc(output)
    gan=Model(input,output2)

    gan.compile(Adam(lr=0.0002),loss=['binary_crossentropy','mse'],metrics=['accuracy'])

    return gan

### Discriminator

In [11]:
def build_sdiscriminator():
    
    input2 = Input(shape=(256,),name='input')
    inp=Dense(64)(input2)

    leaky_relu = LeakyReLU(alpha=0.2)(inp)
    
    conv3 = Dense(128)(leaky_relu)
    b_n = BatchNormalization()(conv3)
    leaky_relu = LeakyReLU(alpha=0.2)(b_n)
    conv4 = Dense(256)(leaky_relu)
    b_n = BatchNormalization()(conv4)
    leaky_relu = LeakyReLU(alpha=0.2)(b_n)
    conv4 = Dense(512)(leaky_relu)
    b_n = BatchNormalization()(conv4)
    leaky_relu = LeakyReLU(alpha=0.2)(b_n)
    conv4 = Dense(1024)(leaky_relu)
    b_n = BatchNormalization()(conv4)
    leaky_relu = LeakyReLU(alpha=0.2)(b_n)

    dense = Dense(1,activation='sigmoid',name='dense')(leaky_relu)

    output2=Dense(256)(leaky_relu)

    
    disc = Model(input2,[dense,output2])          
    disc.compile(optd,loss=['binary_crossentropy','mse'],metrics=['accuracy'])


    return disc

In [12]:
optd = Adam(lr=0.0002)
opt = Adam(lr=0.0002)

In [13]:
d1= build_sdiscriminator()
d1.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 256)]        0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 64)           16448       input[0][0]                      
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 64)           0           dense_1[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 128)          8320        leaky_re_lu[0][0]                
____________________________________________________________________________________________

### Generator

In [14]:
def build_sgenerator(name):

    student1 = Sequential()
    student1.add(Conv2D(64, kernel_size=(3, 3),activation='relu',input_shape=(28, 28, 1),kernel_initializer='normal', name=name))
    student1.add(Conv2D(64, (3, 3), activation='relu',kernel_initializer='normal'))
    student1.add(MaxPooling2D(pool_size=(2, 2)))

    student1.add(Conv2D(16, kernel_size=(3, 3),activation='relu',kernel_initializer='normal'))
    student1.add(Conv2D(16, (3, 3), activation='relu',kernel_initializer='normal'))
    student1.add(MaxPooling2D(pool_size=(2, 2)))

    student1.add(Dropout(0.25)) # For reguralization
    student1.add(Flatten())
    student1.add(Dense(32, activation='relu'))
    student1.add(Dropout(0.3))
    student1.add(Dense(256,name='req'+name))

    student1.compile(opt,loss='mean_squared_error',metrics=['accuracy'])
    
    return student1

build_sgenerator('s').summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
s (Conv2D)                   (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 16)        9232      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 8, 16)          2320      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 16)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 4, 4, 16)         

In [15]:
def training(generator,discriminator,gan,features,epo=20):
    # Setup Models here
    BATCH_SIZE = 128
    discriminator.trainable = True
    total_size = X_train.shape[0]
    indices = np.arange(0,total_size ,BATCH_SIZE)
    all_disc_loss = []
    all_gen_loss = []
    all_class_loss=[]
    if total_size % BATCH_SIZE:
        indices = indices[:-1]
    for e in range(epo):
        
        progress_bar = Progbar(target=len(indices))
        np.random.shuffle(indices)
        epoch_gen_loss = []
        epoch_disc_loss = []
        epoch_class_loss= []
        for i,index in enumerate(indices):
        
            # Split
            inputs=X_train[index:index+BATCH_SIZE]
            sXtrain = features[index:index+BATCH_SIZE]

            y_real = np.ones((BATCH_SIZE,1))
            y_fake = np.zeros((BATCH_SIZE,1))

            #Generator Training
            fake_images = generator.predict_on_batch(inputs)

            #Disrciminator Training
            disc_real_loss1,_,disc_real_loss2,_,_= discriminator.train_on_batch(sXtrain,[y_real,sXtrain])
            disc_fake_loss1,_,disc_fake_loss2,_,_= discriminator.train_on_batch(fake_images,[y_fake,sXtrain])

            #Gans Training
            discriminator.trainable = False
            gan_loss,_,gan_loss2,_,_ = gan.train_on_batch(inputs, [y_real,sXtrain])

            discriminator.trainable = True

            disc_loss = (disc_fake_loss1 + disc_real_loss1)/2
            epoch_disc_loss.append(disc_loss)
            progress_bar.update(i+1)

            epoch_gen_loss.append((gan_loss))

        avg_epoch_disc_loss = np.array(epoch_disc_loss).mean()
        avg_epoch_gen_loss = np.array(epoch_gen_loss).mean()
        all_disc_loss.append(avg_epoch_disc_loss)
        all_gen_loss.append(avg_epoch_gen_loss)
        print("Epoch: %d | Discriminator Loss: %f | Generator Loss: %f | " % (e+1,avg_epoch_disc_loss,avg_epoch_gen_loss))

    return generator

You can initialize multiple instances here 

In [16]:
discriminator1 = build_sdiscriminator()


In [17]:
s1=build_sgenerator("s1")


In [18]:
gan1 = build_gan(s1,discriminator1)


In [19]:
s1 = training(s1,discriminator1,gan1,s1Train,epo=40)

375/375 [==============================] - 17s 41ms/step
Epoch: 1 | Discriminator Loss: 0.582220 | Generator Loss: 1.539414 | 
375/375 [==============================] - 16s 42ms/step
Epoch: 2 | Discriminator Loss: 0.227771 | Generator Loss: 0.882904 | 
375/375 [==============================] - 16s 42ms/step
Epoch: 3 | Discriminator Loss: 0.181847 | Generator Loss: 0.826187 | 
375/375 [==============================] - 15s 41ms/step
Epoch: 4 | Discriminator Loss: 0.160961 | Generator Loss: 0.825641 | 
375/375 [==============================] - 15s 40ms/step
Epoch: 5 | Discriminator Loss: 0.149631 | Generator Loss: 0.798059 | 
375/375 [==============================] - 16s 43ms/step
Epoch: 6 | Discriminator Loss: 0.141141 | Generator Loss: 0.756727 | 
375/375 [==============================] - 15s 41ms/step
Epoch: 7 | Discriminator Loss: 0.131618 | Generator Loss: 0.706982 | 
375/375 [==============================] - 15s 39ms/step
Epoch: 8 | Discriminator Loss: 0.124924 | Generator Lo

### Single Student Configuration

In [20]:
o1=s1.get_layer("reqs1").output

output=Activation('relu')(o1)
output2=Dropout(0.5)(output)
output3=Dense(10,activation="softmax", name="d1")(output2)
mm1=Model(s1.get_layer('s1').input, output3)
my_weights=teacher.get_layer('dense_2').get_weights()

In [21]:
# setting weights
mm1.get_layer('d1').set_weights(my_weights)

In [22]:
# Freezing layers
i=0
for l in mm1.layers[:len(mm1.layers)-1]:
    l.trainable=False
#     print(l)

In [23]:
opt = Adam(lr=0.0002, beta_1=0.5)
mm1.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [24]:
batch_size = 128
mm1.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=20,
          verbose=1, validation_data=(X_val, Y_val))

Epoch 1/20
375/375 [==============================] - 2s 6ms/step - loss: 2.3370 - accuracy: 0.6415 - val_loss: 0.0844 - val_accuracy: 0.9802
Epoch 2/20
375/375 [==============================] - 2s 6ms/step - loss: 0.4191 - accuracy: 0.8946 - val_loss: 0.0743 - val_accuracy: 0.9819
Epoch 3/20
375/375 [==============================] - 2s 5ms/step - loss: 0.3218 - accuracy: 0.9125 - val_loss: 0.0649 - val_accuracy: 0.9834
Epoch 4/20
375/375 [==============================] - 2s 5ms/step - loss: 0.2690 - accuracy: 0.9213 - val_loss: 0.0617 - val_accuracy: 0.9832
Epoch 5/20
375/375 [==============================] - 2s 5ms/step - loss: 0.2380 - accuracy: 0.9301 - val_loss: 0.0596 - val_accuracy: 0.9838
Epoch 6/20
375/375 [==============================] - 2s 4ms/step - loss: 0.2158 - accuracy: 0.9352 - val_loss: 0.0584 - val_accuracy: 0.9833
Epoch 7/20
375/375 [==============================] - 2s 5ms/step - loss: 0.2133 - accuracy: 0.9351 - val_loss: 0.0600 - val_accuracy: 0.9837
Epoch 

In [25]:
loss, acc = mm1.evaluate(X_test, Y_test)

313/313 [==============================] - 1s 4ms/step - loss: 0.0537 - accuracy: 0.9852


In [27]:
mm1.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
s1_input (InputLayer)        [(None, 28, 28, 1)]       0         
_________________________________________________________________
s1 (Conv2D)                  (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 24, 24, 64)        36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 10, 10, 16)        9232      
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 8, 8, 16)          2320      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 4, 4, 16)          0   